In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from scipy.stats.mstats import winsorize

In [2]:
df = pd.read_csv('train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df = df.drop(['MiscFeature', 'Alley', 'PoolQC', 'Fence','LotFrontage', 'FireplaceQu' ], axis = 1)

KeyError: "['MiscFeature' 'Alley' 'PoolQC' 'Fence' 'LotFrontage' 'FireplaceQu'] not found in axis"

In [6]:
df = df.dropna()

#### Veri incelenerek 'SalePrice' değişkenini etkilediği düşünülen 3 farklı parametre seçildi ve yeni bir data frame oluşturuldu.¶

In [11]:
ndf = df[['Street', 'Utilities', 'YearBuilt', 'SalePrice']]

#### Aşağıdaki işlemlerde bir takım metodlarla veri içeriği yeninden ölçeklendirildi ve aykırı değerlerden kurtarıldı.

In [12]:
ndf.YearBuilt = ndf.YearBuilt/1880

c:\users\ömer\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [14]:
x_array = np.array(ndf.SalePrice)
x_norm = preprocessing.normalize([x_array])
x = x_norm.T
ndf.SalePrice = x
ndf.head()

,Street,Utilities,YearBuilt,SalePrice
916,Pave,AllPub,1.036702,0.004762
30,Pave,AllPub,1.021277,0.005394
812,Grvl,AllPub,1.038298,0.007550
1380,Pave,AllPub,1.018085,0.007888
1100,Pave,AllPub,1.021277,0.008091


In [15]:
ndf.SalePrice = winsorize(ndf.SalePrice, 0.045)
ndf.YearBuilt = winsorize(ndf.YearBuilt, 0.007)

In [18]:
ndf["Street"] = ndf.Street.replace({'Pave':1, "Grvl":0})
ndf.Utilities = ndf.Utilities.replace({'AllPub':1, 'NoSeWa':0})

#### Yeni veri kümemize son halini verdik. 

In [20]:
ndf.head()

,Street,Utilities,YearBuilt,SalePrice
916,1,1,1.036702,0.013296
30,1,1,1.021277,0.013296
812,0,1,1.038298,0.013296
1380,1,1,1.018085,0.013296
1100,1,1,1.021277,0.013296


#### OLS kullanılarak modelin OLS özeti çıkarıldı.

In [21]:
import statsmodels.api as sm
Y = ndf.SalePrice
X = ndf[['Street', 'Utilities', 'YearBuilt']]
X = sm.add_constant(X)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                     194.4
Date:                Sun, 17 Mar 2019   Prob (F-statistic):          1.37e-104
Time:                        18:23:12   Log-Likelihood:                 4686.5
No. Observations:                1338   AIC:                            -9365.
Df Residuals:                    1334   BIC:                            -9344.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3043      0.015    -19.732      0.000      -0.335      -0.274
Street         0.0024      0.003      0.742      0.458      -0.004       0.009
Utilities      0.0038      0.007      0.517      0.605      -0.011       0.018
YearBuilt      0.3076      0.013     24.081      0.000       0.283       0.333
==============================================================================
Omnibus:                      205.014   Durbin-Watson:                   0.530
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              310.226
Skew:                           1.071   Prob(JB):                     4.32e-68
Kurtosis:                       3.989   Cond. No.                         197.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""